# DeepFabric ile Watsonx API Kullanarak Sentetik Veri Üretimi

Bu notebook, Watsonx platformundaki Llama modelinizi API üzerinden kullanarak DeepFabric ile sentetik veri üretmeyi gösterir.

## DeepFabric Nedir?

DeepFabric, AI modelleri için sentetik eğitim verisi oluşturan bir kütüphanedir:

- **Konu Ağacı**: Bir konudan alt konular oluşturur
- **Soru-Cevap Üretimi**: Her konu için eğitim verileri üretir
- **Çoklu Format**: TRL, ChatML, Alpaca formatlarını destekler
- **OpenAI-Uyumlu API**: Herhangi bir OpenAI-compatible API ile çalışır

---

## 1. Kurulum

In [ ]:
# DeepFabric kütüphanesini kur
!pip install deepfabric -q

print("✓ DeepFabric kuruldu")

In [ ]:
# Gerekli kütüphaneler
import os
import json
from pathlib import Path

## 2. Watsonx API Konfigürasyonu

### Watsonx API Bilgileriniz:

Eğer Watsonx API'niz OpenAI-uyumlu ise (örneğin bir proxy üzerinden erişiyorsanız), sadece 3 bilgi gerekir:

1. **API URL**: Watsonx API endpoint'iniz
2. **API Key**: Authentication için key (veya dummy key)
3. **Model Adı**: Kullandığınız model adı

DeepFabric, OpenAI formatında API bekliyor. Eğer Watsonx API'niz bu formatta ise direkt kullanabilirsiniz.

In [ ]:
# ========================================
# WATSONX API KONFİGÜRASYONU
# Kendi bilgilerinizi buraya girin
# ========================================

# 1. API URL (örnek: http://your-watsonx-api.com:8000/v1)
API_URL = "http://localhost:8000/v1"  # KENDİ URL'NİZİ GİRİN

# 2. API Key (gerekiyorsa)
API_KEY = "your-api-key-here"  # KENDİ KEY'İNİZİ GİRİN (veya dummy key)

# 3. Model adı
MODEL_NAME = "llama-3-70b-instruct"  # KENDİ MODEL ADINIZI GİRİN

# Environment variables olarak ayarla (DeepFabric bunları kullanacak)
os.environ['OPENAI_API_KEY'] = API_KEY
os.environ['OPENAI_BASE_URL'] = API_URL

print("✅ API Konfigürasyonu tamamlandı")
print(f"  URL: {API_URL}")
print(f"  Model: {MODEL_NAME}")

## 3. API Bağlantısını Test Et (Opsiyonel)

API'nizin çalıştığından emin olmak için basit bir test yapabilirsiniz:

In [ ]:
# OpenAI client ile test (DeepFabric aynı şekilde bağlanır)
try:
    from openai import OpenAI
    
    client = OpenAI(
        api_key=API_KEY,
        base_url=API_URL
    )
    
    # Test prompt
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": "Python nedir?"}],
        max_tokens=100
    )
    
    print("✅ API bağlantısı başarılı!")
    print(f"\nTest yanıtı:\n{response.choices[0].message.content}")
    
except Exception as e:
    print(f"❌ API bağlantı hatası: {e}")
    print("\nLütfen API URL, Key ve Model adını kontrol edin.")

## 4. Basit Örnek: CLI ile Sentetik Veri Üretimi

DeepFabric'in en basit kullanımı komut satırından:

In [ ]:
# Basit bir sentetik veri seti oluştur
!deepfabric generate \
  --mode tree \
  --provider openai \
  --model {MODEL_NAME} \
  --depth 2 \
  --degree 2 \
  --num-steps 10 \
  --topic-prompt "Python Programlama Temelleri" \
  --dataset-save-as dataset_basit.jsonl

print("\n✅ Veri üretimi tamamlandı!")

In [ ]:
# Üretilen veriyi incele
if os.path.exists('dataset_basit.jsonl'):
    print("📊 ÜRETİLEN VERİ ÖRNEKLERİ:\n")
    print("=" * 80)
    
    with open('dataset_basit.jsonl', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i < 2:  # İlk 2 örneği göster
                data = json.loads(line)
                print(f"\n🔹 Örnek {i+1}:")
                print("-" * 80)
                print(json.dumps(data, indent=2, ensure_ascii=False))
                print("=" * 80)
else:
    print("❌ Veri seti bulunamadı")

## 5. TXT Dosyasından Soru-Cevap Üretimi

Kendi txt dosyanızdan sentetik veri oluşturabilirsiniz:

In [ ]:
# Örnek txt dosyası oluştur (veya kendi dosyanızı kullanın)
ornek_metin = """
Yapay Zeka ve Makine Öğrenmesi

Makine öğrenmesi, yapay zekanın bir alt dalıdır ve bilgisayarların 
veriden öğrenmesini sağlar. Temel kavramlar:

1. Supervised Learning: Etiketli veri ile öğrenme
2. Unsupervised Learning: Etiketsiz veri ile öğrenme
3. Reinforcement Learning: Ödül-ceza sistemi ile öğrenme
4. Deep Learning: Çok katmanlı sinir ağları

Model eğitimi için veri kalitesi çok önemlidir.
"""

# Kendi dosyanızı kullanmak için:
# dosya_yolu = "kendi_dosyaniz.txt"

with open('ornek_konu.txt', 'w', encoding='utf-8') as f:
    f.write(ornek_metin)

print("✓ Örnek txt dosyası oluşturuldu: ornek_konu.txt")

In [ ]:
# Txt dosyasını oku
with open('ornek_konu.txt', 'r', encoding='utf-8') as f:
    txt_icerik = f.read()

print(f"📖 Dosya okundu ({len(txt_icerik)} karakter)")
print(f"\nİçerik:\n{txt_icerik[:200]}...")

In [ ]:
# YAML konfigürasyonu oluştur (txt içeriğini topic olarak kullan)
yaml_config = f"""
dataset_system_prompt: |
  Sen yardımcı bir Türkçe AI asistanısın. Kullanıcılara detaylı 
  ve anlaşılır şekilde yanıt veriyorsun.

topic_tree:
  # Txt dosyasının içeriğini ana konu olarak kullanıyoruz
  topic_prompt: |
    {txt_icerik}
    
    Yukarıdaki konular hakkında detaylı alt başlıklar oluştur.
  
  provider: "openai"
  model: "{MODEL_NAME}"
  temperature: 0.7
  degree: 3  # Her konudan 3 alt konu
  depth: 2   # 2 seviye derinlik

data_engine:
  generation_system_prompt: |
    Verilen konular hakkında Türkçe soru-cevap çiftleri oluştur.
    Her cevap:
    - Net ve anlaşılır olmalı
    - Örnekler içermeli  
    - Eğitici olmalı
  
  temperature: 0.6
  provider: "openai"
  model: "{MODEL_NAME}"

dataset:
  creation:
    num_steps: 20      # 20 soru-cevap çifti
    batch_size: 2      # 2'şer 2'şer üret
    sys_msg: true
  template: "builtin://chatml.py"
"""

with open('txt_config.yaml', 'w', encoding='utf-8') as f:
    f.write(yaml_config)

print("✓ YAML konfigürasyon oluşturuldu: txt_config.yaml")

In [ ]:
# Txt dosyasından soru-cevap üret
!deepfabric generate \
  --config txt_config.yaml \
  --dataset-save-as dataset_from_txt.jsonl

print("\n✅ TXT dosyasından sentetik veri üretimi tamamlandı!")

In [ ]:
# Üretilen soru-cevapları incele
if os.path.exists('dataset_from_txt.jsonl'):
    print("\n📊 TXT'DEN ÜRETİLEN SORU-CEVAPLAR:\n")
    print("=" * 80)
    
    with open('dataset_from_txt.jsonl', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i < 3:  # İlk 3 örneği göster
                data = json.loads(line)
                print(f"\n🔹 Örnek {i+1}:")
                print("-" * 80)
                
                if 'messages' in data:
                    for msg in data['messages']:
                        role = msg.get('role', '').upper()
                        content = msg.get('content', '')
                        print(f"\n[{role}]")
                        print(content[:500])
                        if len(content) > 500:
                            print("...")
                
                print("\n" + "=" * 80)
else:
    print("❌ Veri seti bulunamadı")

## 6. Parametrelerin Açıklaması

### Konu Ağacı:

- **`depth`**: Ağaç derinliği (1-5). `depth=2` → Ana konu → Alt konu
- **`degree`**: Her konudan kaç alt konu (1-10). `degree=3` → 3 alt konu
- **`topic_prompt`**: Ana konu metni (txt dosyanızın içeriği olabilir)

### Veri Üretimi:

- **`num_steps`**: Toplam kaç örnek üretilecek
- **`batch_size`**: Paralel üretim için batch sayısı
- **`temperature`**: 0.0 (deterministik) - 1.0 (yaratıcı)

### Prompt'lar:

- **`dataset_system_prompt`**: Sistemin genel davranışı
- **`generation_system_prompt`**: Veri üretim stili
- **`topic_prompt`**: Ana konu (buraya txt içeriğinizi koyun)

---

## 7. Farklı Formatlara Dönüştürme

In [ ]:
# ChatML formatına çevir
!deepfabric format \
  --dataset dataset_from_txt.jsonl \
  --format builtin://chatml.py \
  --output dataset_chatml.jsonl

print("✓ ChatML formatına dönüştürüldü")

In [ ]:
# Alpaca formatına çevir  
!deepfabric format \
  --dataset dataset_from_txt.jsonl \
  --format builtin://alpaca.py \
  --output dataset_alpaca.jsonl

print("✓ Alpaca formatına dönüştürüldü")

In [ ]:
# HuggingFace TRL formatı
!deepfabric format \
  --dataset dataset_from_txt.jsonl \
  --format builtin://trl_sft_tools \
  --output dataset_hf.jsonl

print("✓ HuggingFace TRL formatına dönüştürüldü")

## 8. Veri Analizi Fonksiyonu

In [ ]:
def veri_analizi(dosya_yolu):
    """
    JSONL veri setini analiz eder
    """
    if not os.path.exists(dosya_yolu):
        print(f"❌ Dosya bulunamadı: {dosya_yolu}")
        return
    
    veriler = []
    with open(dosya_yolu, 'r', encoding='utf-8') as f:
        for line in f:
            veriler.append(json.loads(line))
    
    print(f"\n📊 Veri Analizi: {dosya_yolu}")
    print("=" * 60)
    print(f"Toplam örnek: {len(veriler)}")
    
    if veriler and 'messages' in veriler[0]:
        mesaj_sayilari = [len(v['messages']) for v in veriler if 'messages' in v]
        print(f"Ortalama mesaj sayısı: {sum(mesaj_sayilari) / len(mesaj_sayilari):.1f}")
        print(f"Min-Max mesaj: {min(mesaj_sayilari)} - {max(mesaj_sayilari)}")
    
    if veriler:
        print(f"\nVeri yapısı: {list(veriler[0].keys())}")
    
    return veriler

# Kullanım:
# veri_analizi('dataset_from_txt.jsonl')

## 9. Özet

### ✅ Ne Yaptık?

1. DeepFabric kurduk
2. Watsonx API'nizi yapılandırdık
3. Basit veri setleri oluşturduk
4. TXT dosyasından soru-cevap ürettik
5. Farklı formatlara dönüştürdük

### 🎯 Nasıl Kullanırsınız?

**Adım 1:** API bilgilerinizi girin (Bölüm 2)
```python
API_URL = "http://your-api-url/v1"
API_KEY = "your-key"
MODEL_NAME = "your-model"
```

**Adım 2:** Kendi txt dosyanızı kullanın (Bölüm 5)
```python
with open('kendi_dosyaniz.txt', 'r') as f:
    txt_icerik = f.read()
```

**Adım 3:** YAML config oluşturun ve üretin
```bash
deepfabric generate --config txt_config.yaml --dataset-save-as sonuc.jsonl
```

### 💡 İpuçları:

- Küçük başlayın: `num_steps=5-10`
- Spesifik konular seçin
- Prompt'ları özelleştirin
- Temperature ayarlayın (teknik: 0.3-0.5, yaratıcı: 0.7-0.8)

---

**Başarılar!** 🚀